In [ ]:
import os
import cv2
import torch

def process_folder_through_cnn(folder_path, model, batch_size=64, device='cuda'):
    """
    Reads images from a folder, cuts them into 50x50 patches (preserving edges),
    and runs them through a provided CNN model in batches.
    """
    # 1. Setup
    model = model.to(device)
    model.eval() # Set model to evaluation mode (disables dropout, etc.)

    # Get all valid image files in the folder
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                   if f.lower().endswith(valid_extensions)]

    all_results = []
    patch_size = 50

    # Disable gradient tracking to save memory and speed up inference
    with torch.no_grad():
        for img_path in image_files:
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Convert BGR (OpenCV default) to RGB (Standard for CNNs)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            h, w, _ = img.shape

            # 2. Calculate coordinates (no missed pixels, snaps to the edge)
            y_coords = sorted(list(set(list(range(0, max(1, h - patch_size), patch_size)) + [max(0, h - patch_size)])))
            x_coords = sorted(list(set(list(range(0, max(1, w - patch_size), patch_size)) + [max(0, w - patch_size)])))

            patches = []

            # 3. Cut into 50x50 patches
            for y in y_coords:
                for x in x_coords:
                    patch = img[y:y+patch_size, x:x+patch_size]
                    # Ensure patch is exactly 50x50 (skips if original image is smaller than 50x50)
                    if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                        patches.append(patch)

            if not patches:
                continue

            # 4. Prepare for the CNN (Convert to Tensor: Batch, Channels, Height, Width)
            # PyTorch expects dimensions to be (N, C, H, W) and normalized to [0, 1]
            import numpy as np
            patches_array = np.array(patches)
            patches_tensor = torch.from_numpy(patches_array).permute(0, 3, 1, 2).float() / 255.0

            # 5. Run through the model in chunks (batches) to prevent GPU out-of-memory errors
            image_results = []
            for i in range(0, len(patches_tensor), batch_size):
                batch = patches_tensor[i : i + batch_size].to(device)

                # Pass batch through your existing CNN
                output = model(batch)

                # Move predictions back to CPU memory immediately so GPU RAM doesn't fill up
                image_results.append(output.cpu())

            # Store results for this specific image
            all_results.append({
                'image_path': img_path,
                'predictions': torch.cat(image_results, dim=0) # Combines all batches back into one tensor
            })

    return all_results

# --- Example Usage ---
# my_existing_model = MyCNN()
# my_existing_model.load_state_dict(torch.load('weights.pth'))
# results = process_folder_through_cnn('/path/to/images', my_existing_model, batch_size=128, device='cuda')